In [1]:
from load_data_json import LoadDataset 
from preprocess import VectorSpaceModel
import numpy as np
import time
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds, eigsh

## Load the dataset
Functions to load the dataset and the given queries with associated relevant documents.

In [2]:
dataset = LoadDataset("./data/med/med.json", "./data/med/queries.json", "./data/med/qrels-treceval.txt")

In [3]:
dataset.query_vectors.shape

(13004, 30)

# Vector Space Model ranked retrieval

Create the `vsm()` object.

In [4]:
vsm = VectorSpaceModel(dataset.doc_matrix)

In [6]:
vsm.parallel_sci(dataset.query_vectors, dc=2)

Starting 2 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

<lambda>:   0%|          | 0/2 [00:00<?, ?tasks/s]

sci_respone:   0%|          | 0/2 [00:00<?, ?tasks/s]

sci_respone:   0%|          | 0/2 [00:00<?, ?tasks/s]

...

sci_respone:   0%|          | 0/2 [00:00<?, ?tasks/s]

sci_respone:   0%|          | 0/2 [00:00<?, ?tasks/s]

sci_respone:   0%|          | 0/2 [00:00<?, ?tasks/s]

sci_respone:   0%|          | 0/2 [00:00<?, ?tasks/s]

Stopping engine(s): 1711923950
engine set stopped 1711923950: {'engines': {'0': {'exit_code': 1, 'pid': 7372, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 16448, 'identifier': '1'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 9704, 'identifier': 'ipcontroller-1711923948-y67w-7576'}
